In [5]:
import re
from collections import defaultdict
def read_and_process_file(filename='input'):
    file = open(f'{filename}_22', 'r')
    lines = [l.rstrip() for l in file.readlines()]
    
    instructions = lines[-1]
    instructions = re.split('(\d+)',instructions)[:-1]
    instructions = [(instructions[i], int(instructions[i+1])) for i in range(0, len(instructions), 2)]
        
    
    num_cols = max(len(x) for x in lines[:-2])
    num_rows = len(lines[:-2])
    board = [[0 for c in range(num_cols)] for r in range(num_rows)]
    
    for r in range(num_rows):
        for i,c in enumerate(lines[r]):
            if c == '.':
                board[r][i] = 1
            elif c == '#':
                board[r][i] = 9
                
    return board, instructions

In [6]:
turning = {
    "" : 0,
    "R": 1,
    "L": -1
}

movement = {
    0: (0,1),
    2: (0,-1),
    3: (-1,0),
    1: (1,0)
}

printable = {
    0: ">",
    1: "v",
    2: "<",
    3: "ˆ"
}

In [7]:
def get_jump_cells_and_walls(board):
    limits = {}
    walls = set()
    num_rows = len(board)
    num_cols = len(board[0])
    
    for r in range(num_rows):
        non_empty_idx = [i for i,v in enumerate(board[r]) if v>0]
        limits[(r,min(non_empty_idx)),2] = ((r,max(non_empty_idx)),2)
        limits[(r,max(non_empty_idx)),0] = ((r,min(non_empty_idx)),0)
        
    for c in range(num_cols):
        non_empty_idx = [i for i,v in enumerate(board) if v[c]>0]
        limits[(min(non_empty_idx), c),3] = ((max(non_empty_idx), c),3)
        limits[(max(non_empty_idx), c),1] = ((min(non_empty_idx), c),1)
        
    for r in range(num_rows):
        for c in range(num_cols):
            if board[r][c] == 9:
                walls.add((r,c))
        
    return limits, set(walls)

def get_start_id(board):
    return (0,min([c for c in range(len(board[0])) if board[0][c] == 1]))

In [8]:
def calculate_result(position, direction):
    return 1000*(position[0]+1) + 4*(position[1]+1) + direction

In [9]:
def get_position_after_step(position, direction, limits, walls):
    _r, _c = position
    new_position = None
    current_move = movement[direction]

    limit_key = (position, direction)
    
    debug = False
    
    if debug:    
        print(f"Position: {position} | direction: {direction} | movement: {current_move}")
        print(f"Key: {limit_key}")
        
    #check if jump to another side
    if limit_key in limits:
        if debug:
            print(f'Jump to another side (direction: {direction})')
        new_position, new_direction = limits[limit_key]
    else:
        if debug:    
            print(f"Normal step")
        new_position, new_direction = (_r+current_move[0],_c+current_move[1]), direction
     
    #if new position is not a stone return it, otherwise do not move
    if new_position in walls:
        return position, direction, True
    else:
        return new_position, new_direction, False

In [10]:
def explore_board(board, instructions, limits, walls):
    current_position = get_start_id(board)
    current_direction = 0
    for current_instruction in instructions:
        current_direction = (current_direction + turning[current_instruction[0]])%4
        for step in range(current_instruction[1]):
            new_position, new_direction, blocked = get_position_after_step(current_position,current_direction, limits, walls)
            if blocked:
                break
            else:
                #print(f"New position: {new_position} | Direction: {current_direction}")
                board[new_position[0]][new_position[1]] = printable[current_direction]
            current_position = new_position
            current_direction = new_direction

    res = calculate_result(current_position, current_direction)
    print(f"Stopped at {(current_position[0]+1, current_position[1]+1)} and the direction is {current_direction}")
    print(f"The final value after exploring the board is {res}")
    return res

#### One star

In [11]:
board, instructions = read_and_process_file('input')
limits, walls = get_jump_cells_and_walls(board)
res = explore_board(board, instructions, limits, walls)

Stopped at (13, 141) and the direction is 2
The final value after exploring the board is 13566


#### Two stars

In [12]:
#Test: 5031

In [13]:
def detect_cubes(board, cube_size):
    cube_id = 1
    cubes = {}
    b = board.copy()
    current_pos = get_start_id(b)
    while len(cubes)<6:
        if board[current_pos[0]][current_pos[1]]!=0:
            cubes[cube_id] = current_pos
            cube_id+=1
        while ((current_pos[1])+cube_size < len(board[0])):
            new_pos = (current_pos[0],current_pos[1]+cube_size)
            if (board[new_pos[0]][new_pos[1]]!=0):
                cubes[cube_id] = new_pos
                cube_id+=1
            current_pos = new_pos
        current_pos = (current_pos[0]+cube_size, 0)
    return cubes

In [14]:
def calculate_cube_neighbors(cubes, cube_size):
    neighbors = defaultdict(dict)
    for cube_id in range(1,7):
        _cur = cubes[cube_id]
        for n_id in range(cube_id+1, 7):
            _next = cubes[n_id]
            if (_cur[0]==_next[0]) and (_cur[1]+cube_size == _next[1]):
                neighbors[cube_id][0] = n_id
                neighbors[n_id][2] = cube_id

            if (_cur[1]==_next[1]) and (_cur[0]+cube_size == _next[0]):
                neighbors[cube_id][1] = n_id
                neighbors[n_id][3] = cube_id
    for _dir in (0,1,2,3):
        for _cur in range(1,7):
            if _dir in neighbors[_cur]:
                # L connections
                _joiner = neighbors[_cur][_dir]
                _joiner_incoming_dir = [k for k,v in neighbors[_joiner].items() if v==_cur][0]

                incoming_shifts = (1,-1)
                for _shift in incoming_shifts:
                    _joiner_outgoing_dir = (_joiner_incoming_dir+_shift)%4
                    if _joiner_outgoing_dir in neighbors[_joiner]:
                        _tail = neighbors[_joiner][_joiner_outgoing_dir]
                        _tail_incoming_dir = [k for k,v in neighbors[_tail].items() if v==_joiner][0]

                        neighbors[_cur][(_dir + (-1)*_shift)%4] = _tail
                        neighbors[_tail][(_tail_incoming_dir + _shift)%4] = _cur
    return neighbors

In [44]:
def get_jump_cells_and_walls_for_a_cube(board, cubes, neighbors, cube_size):
    limits = {}
    walls = set()
    num_rows = len(board)
    num_cols = len(board[0])
    
    cubes_limits = {k:{"r_s": v[0], "r_e": v[0]+cube_size-1, "c_s": v[1], "c_e": v[1]+cube_size-1} for k,v in cubes.items()}
    all_ranges = {}
    for k,v in cubes_limits.items():
        all_ranges[k] = {0: [(r,v["c_e"]) for r in range(v["r_s"], v["r_e"]+1)],
                  1: [(v["r_e"],c) for c in range(v["c_s"], v["c_e"]+1)],
                  2: [(r,v["c_s"]) for r in range(v["r_s"], v["r_e"]+1)],
                  3: [(v["r_s"],c) for c in range(v["c_s"], v["c_e"]+1)],
                 }
    
    for cube_id in range(1,7):
        for _dir in range(4):
            neighbor = neighbors[cube_id][_dir]
            neighbor_dir = ([k for k,v in neighbors[neighbor].items() if v==cube_id][0]+2)%4
            revert=0
            temp = generate_cases(cubes_limits[cube_id], cubes_limits[neighbor], _dir, neighbor_dir, cube_size, revert)
            limits.update(temp)
            
        
    for r in range(num_rows):
        for c in range(num_cols):
            if board[r][c] == 9:
                walls.add((r,c))
        
    return limits, set(walls)

In [26]:
def generate_cases(range_current, range_neighbor, dir_current, dir_neighbor, cube_size, revert=0):
    limits = {}
    diff_dir = (dir_neighbor-dir_current)%4
    
    iter_current_rows = (dir_current%2 == 0)
    current_dim = (range_current["c_e"], range_current["r_e"], range_current["c_s"], range_current["r_s"])[dir_current]
    
    iter_neighbor_rows = (dir_neighbor%2 == 0)
    neighbor_dim = (range_neighbor["c_e"], range_neighbor["r_e"], range_neighbor["c_s"], range_neighbor["r_s"])[(dir_neighbor+2)%4]
    
    for i in range(cube_size):
        if iter_current_rows:
            if iter_neighbor_rows:
                limits[(range_current["r_s"]+i, current_dim),dir_current] = ((range_neighbor["r_s"]+abs(i-revert), neighbor_dim),dir_neighbor)
            else:
                limits[(range_current["r_s"]+i, current_dim),dir_current] = ((neighbor_dim,range_neighbor["c_s"]+abs(i-revert)),dir_neighbor)
        else:
            if iter_neighbor_rows:
                limits[(current_dim,range_current["c_s"]),dir_current] = ((range_neighbor["r_s"]+abs(i-revert), neighbor_dim),dir_neighbor)
            else:
                limits[(current_dim, range_current["c_s"]+i),dir_current] = ((neighbor_dim, range_neighbor["c_s"]+abs(i-revert)),dir_neighbor)
    
    return limits

In [27]:
cube_size = 4
board, instructions = read_and_process_file('test')
cubes = detect_cubes(board, cube_size)
neighbors = calculate_cube_neighbors(cubes, cube_size)


limits, walls = get_jump_cells_and_walls_for_a_cube(board, cubes, neighbors, cube_size)
res = explore_board(board, instructions, limits, walls)

Stopped at (8, 4) and the direction is 2
The final value after exploring the board is 8018


In [34]:
def generate_cases(range_current, range_neighbor, dir_current, dir_neighbor, cube_size, revert=0):
    limits = {}
    diff_dir = (dir_neighbor-dir_current)%4
    
    iter_current_rows = (dir_current%2 == 0)
    current_dim = (range_current["c_e"], range_current["r_e"], range_current["c_s"], range_current["r_s"])[dir_current]
    
    iter_neighbor_rows = (dir_neighbor%2 == 0)
    neighbor_dim = (range_neighbor["c_e"], range_neighbor["r_e"], range_neighbor["c_s"], range_neighbor["r_s"])[(dir_neighbor+2)%4]
    
    
    for i in range(cube_size):
        if iter_current_rows:
            if iter_neighbor_rows:
                limits[(range_current["r_s"]+i, current_dim),dir_current] = ((range_neighbor["r_s"]+abs(i-revert), neighbor_dim),dir_neighbor)
            else:
                limits[(range_current["r_s"]+i, current_dim),dir_current] = ((neighbor_dim,range_neighbor["c_s"]+abs(i-revert)),dir_neighbor)
        else:
            if iter_neighbor_rows:
                limits[(current_dim,range_current["c_s"]),dir_current] = ((range_neighbor["r_s"]+abs(i-revert), neighbor_dim),dir_neighbor)
            else:
                limits[(current_dim, range_current["c_s"]+i),dir_current] = ((neighbor_dim, range_neighbor["c_s"]+abs(i-revert)),dir_neighbor)
    
    return limits

In [731]:
neighbors

defaultdict(dict,
            {1: {1: (4, 0)},
             4: {3: (1, 0), 2: (3, 0), 1: (5, 0)},
             2: {0: (3, 0)},
             3: {2: (2, 0), 0: (4, 0)},
             5: {3: (4, 0), 0: (6, 0)},
             6: {2: (5, 0)}})

In [732]:
new = []

for c_id in range(1,7):
    for option in [(0,1), (0,3), (2,1), (2,3)]:
        #Check L conections
        n = neighbors[c_id]
        if all([(o in n.keys()) for o in option]):
            print()
            print(option)
            n_1 = n[option[0]][0]
            n_2 = n[option[1]][0]
            d_1 = option[0]
            d_2 = option[1]
            print(f"{n_1} - {n_2}   [{c_id}] - {d_1} {d_2}")
            neighbors[n_1][d_2] = (n_2,999)
            neighbors[n_2][d_1] = (n_1,999)
            print(neighbors[4])


(2, 1)
3 - 5   [4] - 2 1
{3: (1, 0), 2: (3, 0), 1: (5, 0)}

(2, 3)
3 - 1   [4] - 2 3
{3: (1, 0), 2: (3, 0), 1: (5, 0)}

(0, 3)
6 - 4   [5] - 0 3
{3: (1, 0), 2: (3, 0), 1: (5, 0), 0: (6, 999)}

(2, 3)
3 - 4   [5] - 2 3
{3: (1, 0), 2: (3, 999), 1: (5, 0), 0: (6, 999)}

(2, 3)
5 - 4   [6] - 2 3
{3: (1, 0), 2: (5, 999), 1: (5, 0), 0: (6, 999)}


In [726]:
neighbors

defaultdict(dict,
            {1: {1: (4, 0), 2: (3, 999)},
             4: {3: (1, 0), 2: (5, 999), 1: (5, 0), 0: (6, 999)},
             2: {0: (3, 0)},
             3: {2: (2, 0), 0: (4, 0), 1: (5, 999), 3: (4, 999)},
             5: {3: (4, 999), 0: (6, 0), 2: (3, 999)},
             6: {2: (5, 0), 3: (4, 999)}})

In [ ]:
left/right -> up/down -> reverse(xxx)
up/down -> up/down -> up/down -> common left and common right
left-right -> left/right -> left/right -> common up and common down

In [ ]:
new = []
triple_neighbors = defaultdict(dict)

#triple connections
for c_id, n in neighbors.items():
    if len()
    for option in [(0,1), (0,3), (2,1), (2,3)]:
        if all([(o in n.keys()) for o in option]):
            print()
            print(option)
            n_1 = n[option[0]][0]
            n_2 = n[option[1]][0]
            d_1 = option[0]
            d_2 = option[1]
            print(f"n1: {n_1} n2: {n_2} d1: {d_1} d2:{d_2}")
            
            cross_neighbors[n_1][d_2] = (n_2,999)
            cross_neighbors[n_2][d_1] = (n_1,999)
            
            #cross_neighbors[n[option[0]][0]][n[option[1]][0]] = (n[option[1]][0], 999)
            #cross_neighbors[n[option[1]][0]][n[option[0]][0]] = (n[option[0]][0], 999)

In [ ]:
new = []
cross_neighbors = defaultdict(dict)

#L connections expand
for c_id, n in neighbors.items():
    for option in [(0,1), (0,3), (2,1), (2,3)]:
        if all([(o in n.keys()) for o in option]):
            print()
            print(option)
            n_1 = n[option[0]][0]
            n_2 = n[option[1]][0]
            d_1 = option[0]
            d_2 = option[1]
            print(f"n1: {n_1} n2: {n_2} d1: {d_1} d2:{d_2}")
            
            cross_neighbors[n_1][d_2] = (n_2,999)
            cross_neighbors[n_2][d_1] = (n_1,999)
            
            #cross_neighbors[n[option[0]][0]][n[option[1]][0]] = (n[option[1]][0], 999)
            #cross_neighbors[n[option[1]][0]][n[option[0]][0]] = (n[option[0]][0], 999)